In [1]:
import os
import cv2
import random
import numpy as np
import pandas as pd
from tqdm import tqdm
import shutil
from shutil import copyfile
import matplotlib.pyplot as plt
from matplotlib import animation, rc
rc('animation', html='jshtml')
from PIL import Image
import ultralytics
from ultralytics import YOLO
ultralytics.checks()

Ultralytics YOLOv8.2.8  Python-3.11.9 torch-2.3.0+cpu CPU (Intel Core(TM) i7-9700 3.00GHz)
Setup complete  (8 CPUs, 63.9 GB RAM, 24.9/111.8 GB disk)


In [2]:
train_path='./datasets/train/'
valid_path='./datasets/valid/'
test_path='./datasets/test/'

In [3]:
import zipfile

def create_zip(source_folder, destination_zip):
    with zipfile.ZipFile(destination_zip, 'w', zipfile.ZIP_DEFLATED) as zip_ref:
        for root, dirs, files in os.walk(source_folder):
            for file in files:
                file_path = os.path.join(root, file)
                zip_ref.write(file_path, arcname=os.path.relpath(file_path, source_folder))

def extract_zip(zip_file, destination_folder):
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        zip_ref.extractall(destination_folder)

source_folder = '/kaggle/input/cows-images-get-annotation-info-from-xml/datasets'
destination_zip = '/kaggle/working/folder.zip'
destination_folder = '/kaggle/working/datasets'

create_zip(source_folder, destination_zip)
extract_zip(destination_zip, destination_folder)     

In [5]:
import yaml

data_yaml = dict(
    train ='train',
    val ='valid',
    test='test',
    nc =1,
    names =['cow']
)

with open('data.yaml', 'w') as outfile:
    yaml.dump(data_yaml, outfile, default_flow_style=True)

In [7]:
names =['cow']
M=list(range(len(names)))
class_map=dict(zip(M,names))

{0: 'cow'}

In [8]:
model = YOLO("yolov8x.pt")

100%|██████████| 131M/131M [00:02<00:00, 66.6MB/s] 


In [ ]:
!yolo task=detect model=yolov8x.pt data=data.yaml epochs=30 imgsz=480

In [ ]:
predict = model.predict(source='./dataset/indian-bovine-breeds/test/Red_Sindhi')

In [75]:
import shutil
import os

def copy_file(old_path, new_path, new_name):
    # Get the extension of the old file
    _, ext = os.path.splitext(old_path)
    
    # Create the new file path with the new name and the old extension
    new_file_path = os.path.join(new_path, new_name + ext)
    
    # Move the old file to the new file path
    shutil.copy(old_path, new_file_path)

In [94]:
def write_multiple_lines(filename, lines):
    with open(filename, 'a') as f:
        f.write('\n'.join(map(str, lines)))
            

In [96]:
labels = []
labels.append([0, 0.1, 0.2, 0.3, 0.4])
labels.append([0, 0.2, 0.3, 0.4, 0.5])
labels.append([0, 0.6, 0.7, 0.8, 0.9])

lines = []
for label in labels:
    line_str = ' '.join(map(str, label))
    lines.append(line_str)

write_multiple_lines('E:\\AI\\freerolls\\dataset\\train\\labels\\test.txt', lines)

In [99]:
import time

# Path
destination_path = "E:\\AI\\freerolls\\dataset\\indian-bovine-breeds"

images_path = "E:\\AI\\freerolls\\dataset\\train\\images"
labels_path = "E:\\AI\\freerolls\\dataset\\train\\labels"

# Counting index
index = 0

for folder_name in os.listdir(destination_path):
    if os.path.isdir(os.path.join(destination_path, folder_name)):
        source = os.path.join(destination_path, folder_name)
        predict = model.predict(source=source, verbose=False)
        
        for result in predict:
            img_path = result.path  # Get the path of the processed image
            img = result.orig_img

            img_height, img_width = img.shape[:2]  # Get image dimensions
            
            boxes = result.boxes
            
            has_cow = False
            cows = []

            for box in boxes:
                x1, y1, x2, y2 = box.xyxyn[0]
                
                # Convert those to float64
                x1, y1, x2, y2 = x1.numpy(), y1.numpy(), x2.numpy(), y2.numpy()
                
                        
                cls = int(box.cls[0])  # Get class ID
                conf = float(box.conf[0])  # Get confidence
                name = model.names[cls]
                label = f"{model.names[cls]} {conf:.2f}"  # Create label text
                
                if cls == 19:
                    # print(img_path, 'cls:', cls, 'conf:', conf, 'label:', label, 'box:', x1, y1, x2, y2)
                    
                    cows.append([x1, y1, x2, y2])
                    
            # Check cows
            if len(cows) > 0:
                current_time_ns = time.time_ns()
                    
                # Convert dec to hex for set as filename
                hex_time = hex(current_time_ns)
                
                # Copy file
                copy_file(img_path, images_path, hex_time)
                
                # Create labels file
                labels = []
                for cow in cows:
                    x1, y1, x2, y2 = cow
                    width = x2 - x1
                    height = y2 - y1
                    
                    line = [index, x1, y1, width, height]
                    line_str = ' '.join(map(str, line))
                    
                    labels.append(line_str)
                    
                # Line
                label_file = os.path.join(labels_path, f"{hex_time}.txt")
                write_multiple_lines(label_file, labels)
                
        index += 1


image 1/99 E:\AI\freerolls\dataset\indian-bovine-breeds\Alambadi\Alambadi_1.png: 384x640 12 persons, 1 motorcycle, 1 stop sign, 6 cows, 446.0ms
image 2/99 E:\AI\freerolls\dataset\indian-bovine-breeds\Alambadi\Alambadi_10.png: 384x640 9 cows, 422.0ms
image 3/99 E:\AI\freerolls\dataset\indian-bovine-breeds\Alambadi\Alambadi_100.jpg: 384x640 1 cow, 387.0ms
image 4/99 E:\AI\freerolls\dataset\indian-bovine-breeds\Alambadi\Alambadi_101.png: 384x640 1 person, 4 cows, 486.0ms
image 5/99 E:\AI\freerolls\dataset\indian-bovine-breeds\Alambadi\Alambadi_11.png: 384x640 1 cow, 491.0ms
image 6/99 E:\AI\freerolls\dataset\indian-bovine-breeds\Alambadi\Alambadi_12.jpg: 256x640 4 cows, 323.0ms
image 7/99 E:\AI\freerolls\dataset\indian-bovine-breeds\Alambadi\Alambadi_13.png: 384x640 1 cow, 393.0ms
image 8/99 E:\AI\freerolls\dataset\indian-bovine-breeds\Alambadi\Alambadi_14.png: 384x640 4 cows, 378.0ms
image 9/99 E:\AI\freerolls\dataset\indian-bovine-breeds\Alambadi\Alambadi_15.jpg: 512x640 1 sheep, 3 cow

In [68]:
train_folder = 'E:\\AI\\freerolls\\dataset\\train'
test_folder = 'E:\\AI\\freerolls\\dataset\\test'
valid_folder = 'E:\\AI\\freerolls\\dataset\\valid'

# Count all images in predict
len(predict)

# # Process results
# for result in predict:
#     img_path = result.path  # Get the path of the processed image
#     img = result.orig_img

#     img_height, img_width = img.shape[:2]  # Get image dimensions
    
#     boxes = result.boxes

#     for box in boxes:
#         x1, y1, x2, y2 = box.xyxyn[0]
        
#         # Convert those to float64
#         x1, y1, x2, y2 = x1.numpy(), y1.numpy(), x2.numpy(), y2.numpy()
        
                
#         cls = int(box.cls[0])  # Get class ID
#         conf = float(box.conf[0])  # Get confidence
#         name = model.names[cls]
#         label = f"{model.names[cls]} {conf:.2f}"  # Create label text
        
#         if cls == 19:
#             print(img_path, 'cls:', cls, 'conf:', conf, 'label:', label, 'box:', x1, y1, x2, y2)

30